In [1]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'caret' was built under R version 3.6.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 3.6.2"
Loading required package: ggplot2

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v purrr   0.3.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.2"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"pack

## Data partition

In [2]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [3]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Models

In [4]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [5]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size=14, mtry=71, splitrule=gini 
- Fold01.Rep1: min.node.size=14, mtry=71, splitrule=gini 
+ Fold01.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
- Fold01.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
+ Fold01.Rep1: min.node.size=17, mtry=75, splitrule=gini 
- Fold01.Rep1: min.node.size=17, mtry=75, splitrule=gini 
+ Fold02.Rep1: min.node.size=14, mtry=71, splitrule=gini 
- Fold02.Rep1: min.node.size=14, mtry=71, splitrule=gini 
+ Fold02.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
- Fold02.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
+ Fold02.Rep1: min.node.size=17, mtry=75, splitrule=gini 
- Fold02.Rep1: min.node.size=17, mtry=75, splitrule=gini 
+ Fold03.Rep1: min.node.size=14, mtry=71, splitrule=gini 
- Fold03.Rep1: min.node.size=14, mtry=71, splitrule=gini 
+ Fold03.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
- Fold03.Rep1: min.node.size=13, mtry=13, splitrule=extratrees 
+ Fold03.Rep1: min.node.size=17, mtr

In [6]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

Registered S3 method overwritten by 'RRF':
  method      from        
  plot.margin randomForest



+ Fold01.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
- Fold01.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
+ Fold01.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
- Fold01.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
+ Fold01.Rep1: mtry=42, coefReg=0.3010, coefImp=0.2469 
- Fold01.Rep1: mtry=42, coefReg=0.3010, coefImp=0.2469 
+ Fold02.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
- Fold02.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
+ Fold02.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
- Fold02.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
+ Fold02.Rep1: mtry=42, coefReg=0.3010, coefImp=0.2469 
- Fold02.Rep1: mtry=42, coefReg=0.3010, coefImp=0.2469 
+ Fold03.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
- Fold03.Rep1: mtry=37, coefReg=0.9315, coefImp=0.5666 
+ Fold03.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
- Fold03.Rep1: mtry=65, coefReg=0.5971, coefImp=0.8911 
+ Fold03.Rep1: mtry=42, coefReg=0.3010, coefImp=0.2469 
- Fold03.Rep1: mtry=42, coefReg=0.3010, coefImp=

## Results

In [7]:
print(model_rf)
plot(model_rf)

Random Forest 

803 samples
 76 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 723, 723, 722, 723, 723, 723, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
  13             13    extratrees  0.7722452  0.5445336
  14             71    gini        0.7559794  0.5119813
  17             75    gini        0.7546684  0.5093594

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 13, splitrule = extratrees
 and min.node.size = 13.


In [8]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

803 samples
 76 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 722, 723, 723, 723, 723, 723, ... 
Resampling results across tuning parameters:

  mtry  coefReg    coefImp    Accuracy   Kappa    
  37    0.9315340  0.5665572  0.7184105  0.4367226
  42    0.3010400  0.2468891  0.6997840  0.3993035
  65    0.5971278  0.8911292  0.6898611  0.3796981

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 37, coefReg = 0.931534
 and coefImp = 0.5665572.


In [9]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
         Min.   1st Qu.  Median      Mean   3rd Qu.  Max. NA's
ranger 0.6625 0.7391006 0.76250 0.7722452 0.8250000 0.875    0
rrf    0.6500 0.6812500 0.71875 0.7184105 0.7530864 0.775    0

Kappa 
        Min.   1st Qu. Median      Mean   3rd Qu. Max. NA's
ranger 0.325 0.4782012 0.5250 0.5445336 0.6500000 0.75    0
rrf    0.300 0.3625000 0.4375 0.4367226 0.5058713 0.55    0


## Prediction

In [10]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      81      40
   NotFast   59     164
                                          
               Accuracy : 0.7122          
                 95% CI : (0.6612, 0.7595)
    No Information Rate : 0.593           
    P-Value [Acc > NIR] : 2.892e-06       
                                          
                  Kappa : 0.3908          
                                          
 Mcnemar's Test P-Value : 0.07044         
                                          
            Sensitivity : 0.5786          
            Specificity : 0.8039          
         Pos Pred Value : 0.6694          
         Neg Pred Value : 0.7354          
             Prevalence : 0.4070          
         Detection Rate : 0.2355          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6912          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5785714            0.8039216            0.6694215 
      Neg Pred Value            Precision               Recall 
           0.7354260            0.6694215            0.5785714 
                  F1           Prevalence       Detection Rate 
           0.6206897            0.4069767            0.2354651 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6912465

In [11]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [12]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 121 controls (TestClasses fast) < 223 cases (TestClasses NotFast).
Area under the curve: 0.7024

In [13]:
pred_rf.prob

fast,NotFast
0.23633639,0.7636636
0.20491715,0.7950829
0.83370165,0.1662983
0.09546667,0.9045333
0.72458342,0.2754166
0.24916253,0.7508375
0.60731814,0.3926819
0.64362913,0.3563709
0.27708653,0.7229135
0.20576087,0.7942391
